# Start of modelling

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA, IncrementalPCA
from sklearn.linear_model import LinearRegression, HuberRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.model_selection import TimeSeriesSplit
from sklearn.pipeline import make_pipeline
from sklearn.cross_decomposition import PLSRegression
from sklearn.ensemble import GradientBoostingRegressor


### Loading data and changing it to be usable

In [5]:
# Read datafile
y = pd.read_csv('Dependent_y.csv', header=0, index_col=0)
X = pd.read_csv('Features_X.csv', header=0, index_col=0)

y.fillna(0, inplace=True)
y.index = pd.to_datetime(y.index)
X.index = pd.to_datetime(X.index)

weights = pd.read_csv('Stocks_weights.csv', header=0)
weights.index = weights['Date']
weights = weights.drop('Date', axis=1)
weights.index = pd.to_datetime(weights.index)

In [31]:
# For small dataset to test methods
eight_pred = ['mvel1', 'beta', 'betasq', 'chmom', 'dolvol', 'idiovol', 'indmom', 'mom1m']
X_eight_pred = X[eight_pred]

## Functions

In [7]:
from sklearn.model_selection import ParameterGrid

def R_oos(num, den):
    R_oos_val = 1 - (np.sum(num)/np.sum(den))
    return R_oos_val

def val_fun(model, params: dict, X_trn, y_trn, X_vld, y_vld, max_iter=10, tol=1e-4):
    best_ros = None
    lst_params = list(ParameterGrid(params))
    no_improvement_count = 0
    for param in lst_params:
        if best_ros == None:
            mod = model().set_params(**param).fit(X_trn, y_trn)
            y_pred = mod.predict(X_vld)
            best_ros = R_oos(y_vld, y_pred)
            best_param = param
        else:
            mod = model().set_params(**param).fit(X_trn, y_trn)
            y_pred = mod.predict(X_vld)
            ros = R_oos(y_vld, y_pred)
            if ros > best_ros:
                best_ros = ros
                best_param = param
                no_improvement_count = 0
            else:
                no_improvement_count += 1
                if no_improvement_count >= max_iter:
                    break
            if abs(ros - best_ros) < tol:
                break
    return best_param

## 1 - OLS(-3)

### Regression with expanding window and with and without Huber loss

In [8]:
def expanding_regression_OLS(Dependent, Predictors, stock_weights, loss = 'OLS', initial_train_years = 18, validation_years = 12, test_years = 1):
    r_port_difference_list = []
    r_port_actual_list =[]
    years = Dependent.index.year.unique()

    for i in range(len(years) - initial_train_years - validation_years):
        start_year = years[i]
        end_train_year = start_year + initial_train_years  # 18 years of training and increasing with 1 year every iteration
        end_validation_year = end_train_year + validation_years  # 12 years of validation
        end_test_year = end_validation_year + test_years  # 1 year of test

        # Creating training, validation and test sets.
        X_train = Predictors[(Predictors.index.year < end_train_year)]
        X_test = Predictors[(Predictors.index.year >= end_validation_year) & (Predictors.index.year < end_test_year)]
        X_val = Predictors[(Predictors.index.year >= end_train_year) & (Predictors.index.year < end_validation_year)]
        y_train = Dependent[(Dependent.index.year < end_train_year)].values.ravel()
        y_test = Dependent[(Dependent.index.year >= end_validation_year) & (Dependent.index.year < end_test_year)].values.ravel()
        y_val = Dependent[(Dependent.index.year >= end_train_year) & (Dependent.index.year < end_validation_year)].values.ravel()
           
        if loss == 'OLS':
            model = LinearRegression()
        elif loss == 'Huber':
            model = HuberRegressor(epsilon = 99.9) # Set the epsilon to 99.9%.
        else:
            raise ValueError("Invlaid loss function. Use OLS or Huber.")
        
        # Training the model
        OLS3 = model.fit(X_train, y_train)

        # Predict returns at stock level
        r_stock_pred = OLS3.predict(X_test)
        
        # Portofolio test
        r_portfolio_test = np.dot(y_test, stock_weights[(stock_weights.index.year >= end_validation_year) & (stock_weights.index.year < end_test_year)])
        
        # Calculate portfolio return prediction using the given stock weights
        r_portfolio_pred = np.dot(r_stock_pred, stock_weights[(stock_weights.index.year >= end_validation_year) & (stock_weights.index.year < end_test_year)])

        # Appending to lists
        r_port_difference_list.append((r_portfolio_test-r_portfolio_pred)**2)
        r_port_actual_list.append(r_portfolio_test**2)
    
    # Calculate Roos
    Model_Roos = R_oos(r_port_difference_list, r_port_actual_list)
       
    return Model_Roos

In [9]:
#OUT-OF-SAMPLE R^2 for OLS-3
OLS_3pred_Roos=expanding_regression_OLS(y,X_3pred,stock_weights=weights)
OLS_3pred_Roos

0.007429295085335963

In [18]:
#OUT-OF-SAMPLE R^2 for OLS-3 with Huber Loss function
OLS_3pred_Roos_H = expanding_regression_OLS(y, X_3pred,stock_weights=weights, loss = 'Huber')
OLS_3pred_Roos_H

0.07175829363270425

## 2 - Dimension Reduction: PCR and PLS

### 2.1 - PCR

In [15]:
def pcr(Dependent, Predictors, stock_weights, initial_train_years=18, validation_years=12, test_years=1):
    
    # Lists to save outcomes.
    component_counts = []
    r_port_difference_list = []
    r_port_actual_list = []

    yrs = Dependent.index.year.unique() # List with all the years. (1957-2016)
    n_components_list = range(1, 7) # To determine the number of components that are tested.
    best_components = None # Initialize and later save the best amount of components.
    best_r2 = -np.inf  # Initialize with negative infinity to find the maximum R-squared

    for i in range(len(yrs) - initial_train_years - validation_years):
        start_year = yrs[i]
        end_train_year = start_year + initial_train_years  # 18 years of training and increasing with 1 year every iteration
        end_validation_year = end_train_year + validation_years  # 12 years of validation
        end_test_year = end_validation_year + test_years  # 1 year of test

        # Creating training, validation and test sets.
        X_train = Predictors[(Predictors.index.year < end_train_year)]
        X_test = Predictors[(Predictors.index.year >= end_validation_year) & (Predictors.index.year < end_test_year)]
        X_val = Predictors[(Predictors.index.year >= end_train_year) & (Predictors.index.year < end_validation_year)]
        y_train = Dependent[(Dependent.index.year < end_train_year)].values.ravel()
        y_test = Dependent[(Dependent.index.year >= end_validation_year) & (Dependent.index.year < end_test_year)].values.ravel()
        y_val = Dependent[(Dependent.index.year >= end_train_year) & (Dependent.index.year < end_validation_year)].values.ravel()

        # Testing the different components in PCA.
        for n_component in n_components_list:

            pca = PCA(n_components=n_component)
            X_train_pca = pca.fit_transform(X_train) 
            X_val_pca = pca.transform(X_val)

            # Fit Linear Regression on the training set
            model = LinearRegression()
            model.fit(X_train_pca, y_train)

            # Predict on the validation set
            y_val_pred = model.predict(X_val_pca)
            r2 = r2_score(y_val, y_val_pred)

            # Update best components if the current number of components yields a higher R-squared
            if r2 > best_r2:
                best_r2 = r2
                best_components = n_component

        # Save the best number of components to the list
        component_counts.append(best_components)

        # Use the best number of components to fit the final model on the combined training and validation sets
        best_pca = PCA(n_components=best_components)
        X_train_pca = best_pca.fit_transform(X_train)
        X_test_pca = best_pca.transform(X_test)
        
        # Best Model
        PCASP500 = LinearRegression()
        PCASP500.fit(X_train_pca, y_train)

        # Predict returns at the stock level
        r_stock_pred = PCASP500.predict(X_test_pca)
        
        # Portfolio test
        r_portfolio_test = np.dot(y_test, stock_weights[(stock_weights.index.year >= end_validation_year) & (stock_weights.index.year < end_test_year)])
        
        # # Calculate portfolio return prediction using the given stock weights
        r_portfolio_pred = np.dot(r_stock_pred, stock_weights[(stock_weights.index.year >= end_validation_year) & (stock_weights.index.year < end_test_year)])

        # Appending Values to lists
        r_port_difference_list.append((r_portfolio_test-r_portfolio_pred)**2)
        r_port_actual_list.append(r_portfolio_test**2)
        
    # Calculate Roos
    Model_Roos = R_oos(r_port_difference_list, r_port_actual_list)

    return Model_Roos

In [ ]:
PCR_Roo2 = pcr(Dependent=y, Predictors=X_eight_pred, stock_weights=weights)
PCR_Roo2

### 2.2 - PLS

In [56]:
def walk_forward_pls(y_variable, x_variables,stock_weights, initial_train_years=18, validation_years=12, test_years=1):
    
    # Lists to save outcomes.
    r_port_difference_list = []
    r_port_actual_list =[]
    component_counts = []

    years = x_variables.index.year.unique()
    n_components_list = range(1, 7) # To determine the number of components that are tested.
    best_components = None # Initialize and later save the best amount of components.
    best_r2 = -np.inf  # Initialize with negative infinity to find the maximum R-squared

    for i in range(len(years) - initial_train_years - validation_years): # i is 0 to the last start_year which is the last year minus the start training years and validation years.
        start_year = years[i]
        end_train_year = start_year + initial_train_years  # 18 years of training and increasing with 1 year every iteration
        end_validation_year = end_train_year + validation_years  # 12 years of validation
        end_test_year = end_validation_year + test_years  # 1 year of test

        # Creating training, validation and test sets.
        X_train = x_variables[(x_variables.index.year < end_train_year)]
        X_test = x_variables[(x_variables.index.year >= end_validation_year) & (x_variables.index.year < end_test_year)]
        X_val = x_variables[(x_variables.index.year >= end_train_year) & (x_variables.index.year < end_validation_year)]
        y_train = y_variable[(y_variable.index.year < end_train_year)].values.ravel()
        y_test = y_variable[(y_variable.index.year >= end_validation_year) & (y_variable.index.year < end_test_year)].values.ravel()
        y_val = y_variable[(y_variable.index.year >= end_train_year) & (y_variable.index.year < end_validation_year)].values.ravel()

        # Testing the different components in PLS.
        for n_component in n_components_list:

            # Train the model once on the training set
            pls = PLSRegression(n_components=n_component)
            pls.fit(X_train, y_train) 

           # Predict on the validation set
            y_val_pred = pls.predict(X_val)
            r2 = r2_score(y_val, y_val_pred)

            # Update best components if the current number of components yields a higher R-squared
            if r2 > best_r2:
                best_r2 = r2
                best_components = n_component

        # Save the best number of components to the list
        component_counts.append(best_components)

        # Use the best number of components to fit the final model 
        best_pls = PLSRegression(n_components=best_components)
        best_pls.fit(X_train, y_train)

        # Evaluate the final model on the test set
        r_stock_pred = best_pls.predict(X_test)
        
        # Portofolio test
        r_portfolio_test = np.dot(y_test, stock_weights[(stock_weights.index.year >= end_validation_year) & (stock_weights.index.year < end_test_year)])
        
        # Calculate portfolio return prediction using the given stock weights
        r_portfolio_pred = np.dot(r_stock_pred.T, stock_weights[(stock_weights.index.year >= end_validation_year) & (stock_weights.index.year < end_test_year)])
       
        # Appending to lists
        r_port_difference_list.append((r_portfolio_test-r_portfolio_pred)**2)
        r_port_actual_list.append(r_portfolio_test**2)
    
    # Calculate Roos
    Model_Roos = R_oos(r_port_difference_list, r_port_actual_list)
       
    return Model_Roos

In [57]:
# Runs PLS and saves the results
r_squared_scores_pls = walk_forward_pls(y, X_eight_pred,stock_weights=weights ,initial_train_years=18, validation_years=12, test_years=1)

In [58]:
# Prints results.
print(r_squared_scores_pls)
#print(component_counts_pls)
#print(np.mean(r_squared_scores_pls))

-0.05519827043915937


## 3 - Random Forest

#### Selecting Subsamples

In [ ]:
rf_pred = ['mom1m', 'dy']
X_red_rf = X[rf_pred]
X_red_rf

#### Random Forest Function

In [ ]:
from sklearn.ensemble import RandomForestRegressor as RF
from datetime import datetime

def Random_F(Dependent, Predictors, stock_weights, initial_train_years=18, validation_years=12, test_years=1):
    yrs = Dependent.index.year.unique()
    r_port_difference_list = []
    r_port_actual_list = []
    tuning_par = {
    'n_estimators': [300],
    'max_depth': [3,6],
    'max_features': [30,50,100],
    'random_state': [12308]
    }
    
    # Now the model runs for every time of the 30 splits and for every possible combination of the tuning parameters.
    # In this case is 30*60 = 1800, but only the best R2 for every split are stored. 
    for i in range(len(yrs) - initial_train_years - validation_years):
        start_year = yrs[i]
        end_train_year = start_year + initial_train_years  # 18 years of training and increasing with 1 year every iteration
        end_validation_year = end_train_year + validation_years  # 12 years of validation
        end_test_year = end_validation_year + test_years  # 1 year of test

        # Creating training, validation and test sets.
        X_train = Predictors[(Predictors.index.year < end_train_year)]
        X_test = Predictors[(Predictors.index.year >= end_validation_year) & (Predictors.index.year < end_test_year)]
        X_val = Predictors[(Predictors.index.year >= end_train_year) & (Predictors.index.year < end_validation_year)]
        y_train = Dependent[(Dependent.index.year < end_train_year)].values.ravel()
        y_test = Dependent[(Dependent.index.year >= end_validation_year) & (Dependent.index.year < end_test_year)].values.ravel()
        y_val = Dependent[(Dependent.index.year >= end_train_year) & (Dependent.index.year < end_validation_year)].values.ravel()
        
        # This part runs the tuning to find the best combination of the tuning parameters for every split
        best_par = val_fun(RF, params=tuning_par, X_trn=X_train, y_trn=y_train, X_vld=X_val, y_vld=y_val)
        
        # Now we test the model
        RF_SP500 = RF(n_estimators=best_par['n_estimators'], max_depth=best_par['max_depth'], max_features=best_par['max_features'], 
               random_state=best_par['random_state']).fit(X_train, y_train)
        
        # Predict returns at the stock level
        r_stock_pred = RF_SP500.predict(X_test)
        
        # Portofolio test
        r_portfolio_test = np.dot(y_test, stock_weights[(stock_weights.index.year >= end_validation_year) & (stock_weights.index.year < end_test_year)])
        
        # Calculate portfolio return prediction using the given stock weights
        r_portfolio_pred = np.dot(r_stock_pred, stock_weights[(stock_weights.index.year >= end_validation_year) & (stock_weights.index.year < end_test_year)])

        # Appending values to lists
        r_port_difference_list.append((r_portfolio_test-r_portfolio_pred)**2)
        r_port_actual_list.append(r_portfolio_test**2)
        
    # Calculate Roos
    Model_Roos = R_oos(r_port_difference_list, r_port_actual_list)
        
    return Model_Roos

In [ ]:
RF_Roo2 = Random_F(Dependent=y,Predictors=X_red_rf, stock_weights=weights)   
RF_Roo2
#(Running time with 2 predictors: 14 minutes)

### Characteristics of OLS-3
OLS-3 includes size = 'mvel1', Book-to-Market = 'bm', momentum = 'mom12m' Does it also include 'mom1m','mom6m','mom12m','mom36m'?

In [6]:
# Create Dataset for OLS-3 (Need to determine the momentum variable!)
three_predictors = ['mvel1', 'bm','mom12m']
X_3pred = X[three_predictors]

### Gradient Boosted Regression Trees

In [ ]:
def GBRT(Dependent, Predictors, initial_train_years=18, validation_years=12, test_years=1):
    yrs = data.index.year.unique()
    r_squared_scores = []
    tuning_par = {
    'n_estimators': range(1, 150),
    'max_depth': range(1,5),
    'learning_rate': range(0,1)
    }
    
    # Now the model runs for every time of the 30 splits and for every possible combination of the tuning parameters.
    for i in range(len(yrs) - initial_train_years - validation_years):
        start_year = yrs[i]
        end_train_year = start_year + initial_train_years  # 18 years of training and increasing with 1 year every iteration
        end_validation_year = end_train_year + validation_years  # 12 years of validation
        end_test_year = end_validation_year + test_years  # 1 year of test

        train_data = data[(data.index.year < end_train_year)]
        val_data = data[(data.index.year >= end_train_year) & (data.index.year < end_validation_year)]
        test_data = data[(data.index.year >= end_validation_year) & (data.index.year < end_test_year)]

        X_train = train_data[Predictors]
        X_val = val_data[Predictors]
        X_test = test_data[Predictors]
        y_train = train_data[Dependent].values.ravel()
        y_val = val_data[Dependent].values.ravel()
        y_test = test_data[Dependent].values.ravel()

        # This part runs the tuning to find the best combination of the tuning parameters for every split
        best_par = val_fun(GradientBoostingRegressor, params=tuning_par, X_trn=X_train, y_trn=y_train, X_vld=X_val, y_vld=y_val)
        
        # Now we test the model
        GBRT_SP500 = GradientBoostingRegressor(n_estimators=best_par['n_estimators'], max_depth=best_par['max_depth'], learning_rate=best_par['learning_rate']).fit(X_train, y_train)

        # Calculate R_squared
        y_test_pred = GBRT_SP500.predict(X_test)
        test_r2 = r2_score(y_test, y_test_pred)
        r_squared_scores.append(test_r2)
        
    return r_squared_scores

In [ ]:
# run function and print resulting R2 value
GBRT_R2 = GBRT(Dependent=y_variable,Predictors=all_predictors)   
print(GBRT_R2)
print(np.mean(GBRT_R2)) 

## 4 - XGBoost

In [ ]:
from xgboost import XGBRegressor

In [ ]:
def XGBoost(Dependent, Predictors, stock_weights, initial_train_years=18, validation_years=12, test_years=1):
    yrs = Dependent.index.year.unique()
    Roos_list = []
    tuning_par = {
    'n_estimators': [500,600,800,1000],
    'max_depth': [1,2],
    'random_state': [12308],
    #'learning_rate': [.01]
    }
    
    # Now the model runs for every time of the 30 splits and for every possible combination of the tuning parameters.
    # In this case is 30*60 = 1800, but only the best R2 for every split are stored. 
    for i in range(len(yrs) - initial_train_years - validation_years):
        start_year = yrs[i]
        end_train_year = start_year + initial_train_years  # 18 years of training and increasing with 1 year every iteration
        end_validation_year = end_train_year + validation_years  # 12 years of validation
        end_test_year = end_validation_year + test_years  # 1 year of test

        # Creating training, validation and test sets.
        X_train = Predictors[(Predictors.index.year < end_train_year)]
        X_test = Predictors[(Predictors.index.year >= end_validation_year) & (Predictors.index.year < end_test_year)]
        X_val = Predictors[(Predictors.index.year >= end_train_year) & (Predictors.index.year < end_validation_year)]
        y_train = Dependent[(Dependent.index.year < end_train_year)].values.ravel()
        y_test = Dependent[(Dependent.index.year >= end_validation_year) & (Dependent.index.year < end_test_year)].values.ravel()
        y_val = Dependent[(Dependent.index.year >= end_train_year) & (Dependent.index.year < end_validation_year)].values.ravel()
        
        # This part runs the tuning to find the best combination of the tuning parameters for every split
        best_par = val_fun(XGBRegressor, params=tuning_par, X_trn=X_train, y_trn=y_train, X_vld=X_val, y_vld=y_val)
        
        # Now we test the model
        XGB = XGBRegressor(n_estimators=best_par['n_estimators'], max_depth=best_par['max_depth']).fit(X_train, y_train)
        
        # Predict returns at the stock level
        r_stock_pred = XGB.predict(X_test)
        
        # Portofolio test
        r_portfolio_test = np.dot(y_test, stock_weights[(stock_weights.index.year >= end_validation_year) & (stock_weights.index.year < end_test_year)])
        
        # Calculate portfolio return prediction using the given stock weights
        r_portfolio_pred = np.dot(r_stock_pred, stock_weights[(stock_weights.index.year >= end_validation_year) & (stock_weights.index.year < end_test_year)])

        # Calculate Roos
        Roos_val = R_oos(r_portfolio_test, r_portfolio_pred)
        Roos_list.append(Roos_val)
        
    return Roos_list

In [ ]:
XGB_Roo2 = XGBoost(Dependent=y,stock_weights=weights,Predictors=X_red_rf)   
XGB_Roo2

In [ ]:
np.mean(XGB_Roo2)